In [2]:
from deltalake import DeltaTable, write_deltalake
import duckdb

con = duckdb.connect()

In [3]:
con.sql("SELECT * FROM 'categories.csv'")

┌─────────────┬─────────────────────┐
│ category_id │    category_name    │
│    int64    │       varchar       │
├─────────────┼─────────────────────┤
│           1 │ Children Bicycles   │
│           2 │ Comfort Bicycles    │
│           3 │ Cruisers Bicycles   │
│           4 │ Cyclocross Bicycles │
│           5 │ Electric Bikes      │
│           6 │ Mountain Bikes      │
│           7 │ Road Bikes          │
└─────────────┴─────────────────────┘

In [4]:
df = con.sql("SELECT * FROM 'categories.csv'").to_arrow_table()

In [5]:
storage_options = {
    "AWS_ACCESS_KEY_ID": "E1bb9GmAwnOnxY3YFdfL",
    "AWS_SECRET_ACCESS_KEY": "9wehFY9StKHamvgfnjWLoPphNOTYTnWIjzNkYpTn",
    "AWS_ENDPOINT_URL": "http://localhost:9002",
    "AWS_allow_http": "true",
    "AWS_REGION": "us-east-1",
    "AWS_S3_ALLOW_UNSAFE_RENAME": "true",
}

In [21]:
con.sql(""" CREATE SECRET secret1 (
            TYPE S3,
            KEY_ID 'E1bb9GmAwnOnxY3YFdfL',
            SECRET '9wehFY9StKHamvgfnjWLoPphNOTYTnWIjzNkYpTn',
            REGION 'us-east-1',
            ENDPOINT 'localhost:9002',
            URL_STYLE 'path',
            USE_SSL 'false'

        );
    """)

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

In [6]:
uri = "s3://land/uff/delta/categories"

write_deltalake(uri, df, mode="append", storage_options=storage_options)

In [7]:
dt = DeltaTable(uri, storage_options=storage_options)

In [8]:
dt.schema().to_pyarrow()

category_id: int64
category_name: string

In [13]:
dt.to_pandas()

,category_id,category_name
0,1,Children Bicycles
1,2,Comfort Bicycles
2,3,Cruisers Bicycles
3,4,Cyclocross Bicycles
4,1,Children Bicycles
5,2,Comfort Bicycles
6,3,Cruisers Bicycles
7,4,Cyclocross Bicycles


In [14]:
dt.delete("category_id > 4")

{'num_added_files': 0,
 'num_removed_files': 0,
 'num_deleted_rows': 0,
 'num_copied_rows': 0,
 'execution_time_ms': 1,
 'scan_time_ms': 0,
 'rewrite_time_ms': 1}

In [16]:
df = con.sql("SELECT * FROM 'categories.csv'").to_arrow_table()

In [17]:
(
    dt.merge(
        df,
        "delta.category_id = arquivo.category_id",
        source_alias="arquivo",
        target_alias="delta",
    )
    .when_matched_update_all()
    .when_not_matched_insert_all()
    .execute()
)

{'num_source_rows': 7,
 'num_target_rows_inserted': 3,
 'num_target_rows_updated': 8,
 'num_target_rows_deleted': 0,
 'num_target_rows_copied': 0,
 'num_output_rows': 11,
 'num_target_files_scanned': 1,
 'num_target_files_skipped_during_scan': 0,
 'num_target_files_added': 2,
 'num_target_files_removed': 1,
 'execution_time_ms': 39,
 'scan_time_ms': 0,
 'rewrite_time_ms': 14}

In [19]:
dt.history()

[{'timestamp': 1732635364612,
  'operation': 'MERGE',
  'operationParameters': {'notMatchedPredicates': '[{"actionType":"insert"}]',
   'mergePredicate': 'delta.category_id = arquivo.category_id',
   'predicate': 'category_id BETWEEN 1 AND 7',
   'notMatchedBySourcePredicates': '[]',
   'matchedPredicates': '[{"actionType":"update"}]'},
  'readVersion': 2,
  'clientVersion': 'delta-rs.0.22.0',
  'operationMetrics': {'execution_time_ms': 39,
   'num_output_rows': 11,
   'num_source_rows': 7,
   'num_target_files_added': 2,
   'num_target_files_removed': 1,
   'num_target_files_scanned': 1,
   'num_target_files_skipped_during_scan': 0,
   'num_target_rows_copied': 0,
   'num_target_rows_deleted': 0,
   'num_target_rows_inserted': 3,
   'num_target_rows_updated': 8,
   'rewrite_time_ms': 14,
   'scan_time_ms': 0},
  'version': 3},
 {'timestamp': 1732634784997,
  'operation': 'DELETE',
  'operationParameters': {'predicate': 'category_id > 4'},
  'readVersion': 1,
  'clientVersion': 'delta-

In [27]:
dados = con.sql(" SELECT * FROM 's3://land/uff/delta/categories/_delta_log/*.json'")

In [31]:
dados["commitInfo"]["operation"].aggregate("operation")

┌───────────┐
│ operation │
│  varchar  │
├───────────┤
│ DELETE    │
│ MERGE     │
│ NULL      │
│ WRITE     │
└───────────┘

In [33]:
dt.version()

3

In [40]:
dt.load_as_version(1)

In [42]:
dt.to_pandas()

,category_id,category_name
0,1,Children Bicycles
1,2,Comfort Bicycles
2,3,Cruisers Bicycles
3,4,Cyclocross Bicycles
4,5,Electric Bikes
5,6,Mountain Bikes
6,7,Road Bikes
7,1,Children Bicycles
8,2,Comfort Bicycles
9,3,Cruisers Bicycles


In [43]:
con.close()